Questions:
1. How are these algorithms sensitive to the Lipschitz constant?
2. What is a good way to choose L? and correspondingly alpha?

Things to Review:
1. Proximal Function and writing that out

# Least Squares Solver
The least squares optimization problem is as follows:

$$J(f(x)) := \frac{1}{2} \|Ax-b\|_2^2$$

which just uses the L2-norm, square of the Ax=b systems of equations, and minimizes that error.
Gradient of cost function:

$$\nabla{f(x)} = A^T(Ax-b)$$

Generally, b are observations and A is some sort of model transforming our unknown parameter space/variables into the observation predictions.

# Gradient Descent Algorithm
To minimize the cost function, J, just compute the gradient of the cost function and step in that direction. For some $\alpha_{k}$, generally minimizing step size, the iteration is computed in a for loop as follows.

$$x_{k+1} = x_{k} - \alpha_{k}\nabla{f(x)}$$

# ISTA
This is similar to the gradient descent algorithm, but is generally used to solve problems that fall in the class of proximal gradient methods. The new cost function is something as follows:

$$J(f(x)) := \frac{1}{2} \|Ax-b\|_2^2 + r(x)$$

where r(x) can be defined as any proximal, nonsmooth convex function. Examples, include l1, l2, or elastic-net proximal terms. For, those the new cost function is as follows:

$$J(f(x)) := \frac{1}{2} \|Ax-b\|_2^2 + \lambda_1\|x\|_1 + \lambda_2\|x\|_2^2$$ or
$$J(f(x)) := \frac{1}{2} \|Ax-b\|_2^2 + (1-\alpha)\|x\|_1 + \alpha\|x\|_2^2$$ 
with 
$$\alpha = \frac{\lambda_2}{\lambda_2+\lambda_1}$$

The derivation of the iterative algorithm comes from first-order optimality conditions - 0 must be in the subdifferential of the cost function. The new cost function gives rise to a subproblem of solving for $x_{k+1}$ in two segments, one for f(x) and one for r(x). So just set 0 = the subdifferential and then solve for $x_{k+1}$

$$x_{k+1} = Shrink(x_{k} - \alpha_{k}\nabla{f(x)};\lambda\alpha_k)$$

# FISTA
This is ISTA, but with the ideas of Nesterov's Acceleration. It has the same general cost functions as ISTA would with some proximal term. However, with an acceleration/extrapolation term, this algorithm performs with O(1/k^2)

$$x_{k} = Shrink(Y_{k} - \alpha_k\nabla(f(Y));\lambda\alpha_k) \ \text{Proximal Step}$$ 

$$t_{k+1} = \frac{1+\sqrt{1+4t_k^2}}{2} \ \text{Compute Step Size}$$

$$y_{k} = x_{k} + \frac{t_k-1}{t_{k+1}})(x_k - x_{k-1}) \ \text{Extrapolation} $$

## Step Size
All algorithms can use some sort of step size that is arbitrary depending on the problem. A common choice can be using a constant step size = 1/L, where L is the Lipschitz constant for the gradient of the cost function.

1. Least Squares
    $$L = \frac{ \|\mathbf{x}^T \mathbf{x}\|_{op}}{n}$$
2. Logistic
    $$L = \frac{\underset{i}{\max}(\|x_i\|_2^2)}{4n}$$


References:
1. Elastic Net: http://web.stanford.edu/~hastie/TALKS/enet_talk.pdf
2. Code: http://nbviewer.jupyter.org/github/zermelozf/notebooks/blob/master/First%20order%20optimization.ipynb#full
https://github.com/JeanKossaifi/FISTA

3. FISTA: http://people.rennes.inria.fr/Cedric.Herzet/Cedric.Herzet/Sparse_Seminar/Entrees/2012/11/12_A_Fast_Iterative_Shrinkage-Thresholding_Algorithmfor_Linear_Inverse_Problems_(A._Beck,_M._Teboulle)_files/Breck_2009.pdf

In [17]:
import numpy as np
from numpy.linalg import norm
import scipy.optimize

%matplotlib inline
import matplotlib.pylab as plt

## Define Cost Functions and Gradients

In [270]:
'''
A = matrix of features n_samples X n_features
x = data = n_features X 1
b = observations n_samples X 1
'''
# def least_squares(x, features, labels):
#     """Evaluates the least square function."""
#     n_samples = features.shape[0]
#     x = x.reshape(1, n_features)
#     loss_array = 1/2*(features.dot(x.T) - labels) ** 2
#     return np.sum(loss_array, axis=0)

# def least_squares_grad(x, features, labels):
#     """Evaluates the gradient of the least square function."""
#     n_samples = features.shape[0]
#     x = x.reshape(1, n_features)  # Added for scipy.optimize compatibility
#     grad_array = (features.dot(x.T) - labels) * features
#     return np.sum(grad_array, axis=0) / n_samples

def least_squares(A, x, b):
    """Evaluates the least square function."""
    n_samples, n_features = A.shape
    x = x.reshape(n_features, 1)
    loss_array = 1/2*(A.dot(x) - b) ** 2
    return np.sum(loss_array, axis=0)

def least_squares_grad(A, x, b):
    """Evaluates the gradient of the least square function."""
    n_samples, n_features = A.shape
    x = x.reshape(n_features, 1)  # Added for scipy.optimize compatibility
    grad_array = (A.dot(x) - b) * A
    return (np.sum(grad_array, axis=0) / n_samples).reshape(n_features, 1)

## Define Proximal Operators

In [271]:
# input: x = xk - step*grad
# x_abs > step*lambda_ = shrinkage operator
def prox_l1(x, step, lambda_):
    """ Proximal operator of the l1 norm."""
    x_abs = np.abs(x) # get the absolute value
    shrink_op = step*lambda_ # alpha_k * lambda_
    return np.sign(x) * (x_abs - shrink_op) * (x_abs > shrink_op)

def prox_l2(x, l=1.):
    """ Proximal operator of the l2 norm."""
    return 1. / (1 + l) * x

def prox_enet(x, l_l1, l_l2, t=1.):
    """Proximal operator for the elastic net at x"""
    x_abs = np.abs(x)
    prox_l1 = np.sign(x) * (x_abs - l_l1) * (x_abs > l_l1)
    return prox_l1 + 1./ (1. + l_l2) * x_abs

In [272]:
def inspector(loss_fun, x_real, verbose=False):
    """A closure called to update metrics after each iteration."""
    objectives = []
    errors = []
    it = [0]  # This is a hack to be able to modify 'it' inside the closure.
    def inspector_cl(xk):
        obj = loss_fun(xk)
        err = norm(xk - x_real) / norm(x_real)
        objectives.append(obj)
        errors.append(err)
        if verbose == True:
            if it[0] == 0:
                print ' | '.join([name.center(8) for name in ["it", "obj", "err"]])
            if it[0] % (n_iter / 5) == 0:
                print ' | '.join([("%d" % it[0]).rjust(8), ("%.2e" % obj).rjust(8), ("%.2e" % err).rjust(8)])
            it[0] += 1
    inspector_cl.obj = objectives
    inspector_cl.err = errors
    return inspector_cl

## Define Algorithms

In [273]:
def grad_descent(x_init, grad, n_iter=100, step=1., tol=None, callback=None):
    x = x_init.copy() # initialize x
    
    # perform iterations
    for _ in range(n_iter):
        x -= step * grad(x)
        
        # update metrics in a function
        if callback is not None:
            callback(x)
            
        # if gradient has reached a low point
        if tol != None and grad(x) <= tol:
            break
            
    return x

In [274]:
def ista(x_init, grad, prox, n_iter, step=1., cost=least_squares, tol=None, callback=None):
    x = x_init.copy() # initialize x
    
    # perform iterations
    for _ in range(n_iter):
        x = prox(x - step * grad(x))
        
        # update metrics in a function
        if callback is not None:
            callback(x)
            
        # if gradient has reached a low point
        if tol != None and cost(x) <= tol:
            break
            
    return x

In [275]:
def fista(x_init, grad, prox, lambda_, n_iter, step=1., cost=least_squares, tol=None, callback=None):
    x = x_init.copy() # initialize x
    y = x_init.copy()
    t = 1.
    
    # perform iterations
    for _ in range(n_iter):
        x_new = prox(y - step * grad(y), step)
        t_new = (1. + (1. + 4. * t**2)**.5) / 2
        y = x_new + (t - 1) / t_new * (x_new - x)
        t = t_new
        x = x_new
        
        # update metrics in a function
        if callback is not None:
            callback(x)
            
        # if gradient has reached a low point
        if tol != None and cost(x) <= tol:
            break
            
    return x

## Simulations

In [276]:
# Generate a fake dataset
n_samples = 2000
n_features = 50

idx = np.arange(n_features).reshape(1, n_features)
params = 2 * (-1) ** (idx - 1) * .9**idx
params[0, 20:50] = 0
params = params.T
diag = np.random.rand(n_features)
A = np.random.multivariate_normal(np.zeros(n_features), np.diag(diag), n_samples)

residuals = np.random.randn(n_samples, 1)
b = A.dot(params) + residuals

# Show the condition number of the gram matrix
print "cond = %.2f" % (diag.max() / diag.min())
print 'features: ', A.shape
print 'real weights: ', params.shape
print 'b: ', b.shape

# Initialize stuff
x_init = 1 - 2 * np.random.rand(n_features, 1)
n_iter = 30
l_l1 = 0.0
l_l2 = 0.1

test = (A.dot(x_init) - b) * A
test = np.sum(test, axis=0)
print test.shape

# f and gradient
f = lambda x: least_squares(A,x, b)
grad_f = lambda x: least_squares_grad(A, x, b)
step = norm(A.T.dot(A) / n_samples, 2)

# g, F and prox.
g = lambda x: l_l1 * np.abs(x).sum()
F = lambda x: f(x) + g(x)
prox_g = lambda x: prox_l1(x, step, l_l1)

print 'prox',prox_g(x_init).shape
print 'grad',grad_f(x_init).shape

print 'x init: ', x_init.shape
print "n_iter: %d" % n_iter
print "step size: %.2f" % step
# Gradient descent
# Gradient descent
grad_gd = lambda x: grad_f(x) + l_l1 * np.abs(x)
print grad_gd(x_init).shape

gd_inspector = inspector(loss_fun=F, x_real=params, verbose=True)
x_gd = grad_descent(x_init, grad=grad_gd, n_iter=n_iter, step=step, callback=gd_inspector)

cond = 18.54
features:  (2000, 50)
real weights:  (50, 1)
b:  (2000, 1)
(50,)
prox (50, 1)
grad (50, 1)
x init:  (50, 1)
n_iter: 30
step size: 1.12
(50, 1)
   it    |   obj    |   err   
       0 | 0.00e+00 | 8.18e-01
       6 | 0.00e+00 | 3.36e-01
      12 | 0.00e+00 | 1.96e-01
      18 | 0.00e+00 | 1.31e-01
      24 | 0.00e+00 | 9.73e-02
